In [1]:
using Revise

┌ Info: Recompiling stale cache file /home/amir/.julia/compiled/v1.1/Revise/M1Qoh.ji for Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1184


In [2]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using ExactDiagonalizationTools
using MatrixProductStateTools
using TensorNetAlgs
using Plots

In [3]:
lx = 41
delta = 2. #ratio of Jz to Jp
r = 0.6
maxdim = 16
dt = 0.05

nsteps = 40
nsamples = 10

10

In [ ]:
mpo = xxzlong_mpo(ComplexF64, lx, 2, delta, r)
mpoc = convert(MatrixProductOperator{ComplexF64}, mpo)
sz = Matrix(sz_half)

alldata = []
for sample in 1:nsamples
    conf = rand(0:1, lx)
    for k in [0,1]
        zdata = Vector{Float64}[]
        conf[div(lx, 2)+1] = k
        mps = MatrixProductState{ComplexF64}(lx, 2, conf, maxdim, 1.0e-6)
        #normalize!(mps)
        push!(zdata, (-1)^k .* real.(measure_1point(mps, sz)))
        println("Sample $sample, spin $k, time : 0.0")

        env = initialenv(mps, mpoc)
        for t in dt:dt:nsteps*dt
            tdvp1sitesweep!(dt, mps, mpoc, env, verbose=false)
            if abs(round(t,digits=1)-t) ≈ 0.
                push!(zdata, (-1)^k .* real.(measure_1point(mps, sz)))
                println("Sample $sample, spin $k, time : $t")
            end
        end
        push!(alldata, zdata)
    end
end

Sample 1, spin 0, time : 0.0
Sample 1, spin 0, time : 0.1
Sample 1, spin 0, time : 0.2
Sample 1, spin 0, time : 0.3
Sample 1, spin 0, time : 0.4
Sample 1, spin 0, time : 0.5
Sample 1, spin 0, time : 0.6
Sample 1, spin 0, time : 0.7
Sample 1, spin 0, time : 0.8
Sample 1, spin 0, time : 0.9
Sample 1, spin 0, time : 1.0
Sample 1, spin 0, time : 1.1
Sample 1, spin 0, time : 1.2
Sample 1, spin 0, time : 1.3
Sample 1, spin 0, time : 1.4
Sample 1, spin 0, time : 1.5
Sample 1, spin 0, time : 1.6
Sample 1, spin 0, time : 1.7
Sample 1, spin 0, time : 1.8
Sample 1, spin 0, time : 1.9
Sample 1, spin 0, time : 2.0
Sample 1, spin 1, time : 0.0
Sample 1, spin 1, time : 0.1
Sample 1, spin 1, time : 0.2
Sample 1, spin 1, time : 0.3
Sample 1, spin 1, time : 0.4
Sample 1, spin 1, time : 0.5
Sample 1, spin 1, time : 0.6
Sample 1, spin 1, time : 0.7
Sample 1, spin 1, time : 0.8
Sample 1, spin 1, time : 0.9
Sample 1, spin 1, time : 1.0
Sample 1, spin 1, time : 1.1
Sample 1, spin 1, time : 1.2
Sample 1, spin

In [ ]:
p=plot()
for t in 1:Int(nsteps*dt)+1
    avgdata = zeros(length(alldata[1][1]))
    for sample in 1:nsamples
        avgdata += 2*alldata[sample][t]
    end
    plot!(p, avgdata)
end
p